In [ ]:
import pandas as pd
from var_names import *
# import numpy as np

In [ ]:
def normalized_value_counts(series):
   """Calculate value counts and percentages for a pandas Series."""
   counts = series.value_counts(dropna=False)
   percentages = series.value_counts(normalize=True, dropna=False).mul(100).round(2).astype(str) + '%'
   # result = pd.concat([counts, percentages], axis=1, keys=['N', '%'])
   result = pd.DataFrame({
      'N': counts,
      '%': percentages
   })
   return result

In [ ]:
# codebook = pd.read_excel('codebook.xlsx')
baseline = pd.read_csv('tba_data.csv')

In [ ]:
baseline

In [ ]:
# processing order
def to_string_list(filepath):
   """Read elements from a file into a list."""
   try:
      with open(filepath, 'r') as file:
         elements = file.read().strip().split('\n')
      return [str(element) for element in elements]
   except IOError as e:
      print(f"An error occurred: {e.strerror}")
      return []

In [ ]:
def remove_sequence(lst1, lst2):
   # Find the start of the matching sequence
   start_index = None
   for i in range(len(lst1) - len(lst2) + 1):
      if lst1[i:i + len(lst2)] == lst2:
         start_index = i
         break
   # If the sequence was found, remove it
   if start_index is not None:
      del lst1[start_index:start_index + len(lst2)]
   return lst1

In [ ]:
order_list = to_string_list('order.txt')

In [ ]:
# order_map

In [ ]:
def tabulation(df, variable, order_list):
   """Tabulate value counts for a specific variable in a DataFrame."""
   series = df[variable]
   counts_order = [x for x in order_list if x in series.unique()]
   counts = normalized_value_counts(series=series).reindex(counts_order)
   new_order_list = remove_sequence(order_list, counts_order)
   return counts, new_order_list

In [ ]:
question = 'qual_life2'
counts, order_list = tabulation(baseline, question, order_list)
# output_file = 'single_variables.xlsx'

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
# with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
#    for var in question:
#       Check if the variable exists in the dataframe
#       if var in baseline.columns:
         # Apply your function to the variable, and get the results
         # result_df = normalized_value_counts(baseline[var])
         # Write each DataFrame to a specific sheet
         # result_df.to_excel(writer, sheet_name=var)
      # else:
      #    print(f"Variable {var} not found in DataFrame.")

In [ ]:
baseline['network_size'].value_counts()

In [ ]:
# baseline['relative_support2'].value_counts(normalize=True) * 100